<a href="https://colab.research.google.com/github/TobyW28/FTL-TRANS/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_csv("/content/drive/My Drive/fmt_data.csv")

In [ ]:
new_column_names = {"HADM_ID": "Adm_ID", "ROW_ID_y": "Note_ID", "CHARTDATE": "chartdate", "CHARTTIME": "charttime", "TEXT": "TEXT", "HOSPITAL_EXPIRE_FLAG": "Label"}
data.rename(columns=new_column_names, inplace=True)

data

,Unnamed: 0,Adm_ID,Note_ID,chartdate,charttime,TEXT,Label
0,0,165315,12144,2196-04-10,NaN,Admission Date: [**2196-4-9**] Discharg...,0
1,1,165315,158364,2196-04-09,NaN,Normal sinus rhythm. Non-diagnostic repolariza...,0
2,2,165315,158365,2196-04-09,NaN,Normal sinus rhythm without diagnostic abnorma...,0
3,3,165315,754366,2196-04-10,2196-04-10 23:52:00,[**2196-4-10**] 11:52 PM\n CHEST (PORTABLE AP)...,0
4,4,165315,754332,2196-04-09,2196-04-09 10:19:00,[**2196-4-9**] 10:19 AM\n CT HEAD W/O CONTRAST...,0
...,...,...,...,...,...,...,...
1851339,1851339,170407,988388,2128-11-16,2128-11-16 11:54:00,[**2128-11-16**] 11:54 AM\n CHEST (PORTABLE AP...,0
1851340,1851340,190264,1229,2131-10-26,NaN,Admission Date: [**2131-10-25**] ...,0
1851341,1851341,190264,1161225,2131-10-25,2131-10-25 00:38:00,[**2131-10-25**] 12:38 AM\n CT HEAD W/O CONTRA...,0
1851342,1851342,190264,1161226,2131-10-25,2131-10-25 00:39:00,[**2131-10-25**] 12:39 AM\n CT L-SPINE W/O CON...,0


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 101.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.9 MB/s eta 0:00:00


In [ ]:
pip install pytorch-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 44.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 109.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.3 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=c789940178c2f849c80b39e6675dc20b3a8f9c4c348f28dbe62b30196a5cfef0
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [ ]:
from tqdm import tqdm, trange
import pandas as pd
import io
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import re
import argparse
from pytorch_transformers import BertTokenizer
from sklearn.model_selection import KFold

def write_log(content, log_path, print_content=True):
    if os.path.exists(log_path):
        with open(log_path, 'a') as f:
            f.write("Time: " + time.ctime() + "\n")
            f.write(content + "\n")
            f.write("=====================\n")
    else:
        with open(log_path, 'w') as f:
            f.write("Time: " + time.ctime() + "\n")
            f.write(content + "\n")
            f.write("=====================\n")
    if print_content:
        print(content)


def preprocess1(x):
    y = re.sub('\\[(.*?)\\]', '', x)  # remove de-identified brackets
    y = re.sub('[0-9]+\.', '', y)  # remove 1.2. since the segmenter segments based on this
    y = re.sub('dr\.', 'doctor', y)
    y = re.sub('m\.d\.', 'md', y)
    y = re.sub('admission date:', '', y)
    y = re.sub('discharge date:', '', y)
    y = re.sub('--|__|==', '', y)
    return y


def preprocessing(df_less_n, tokenizer):
    df_less_n['TEXT'] = df_less_n['TEXT'].fillna(' ')
    df_less_n['TEXT'] = df_less_n['TEXT'].str.replace('\n', ' ')
    df_less_n['TEXT'] = df_less_n['TEXT'].str.replace('\r', ' ')
    df_less_n['TEXT'] = df_less_n['TEXT'].apply(str.strip)
    df_less_n['TEXT'] = df_less_n['TEXT'].str.lower()

    df_less_n['TEXT'] = df_less_n['TEXT'].apply(lambda x: preprocess1(x))

    sen = df_less_n['TEXT'].values
    tokenized_texts = [tokenizer.tokenize(x) for x in sen]
    print("First sentence tokenized")
    print(tokenized_texts[0])
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    df_less_n['Input_ID'] = input_ids
    return df_less_n[["Adm_ID", "Note_ID", "chartdate", "charttime", "TEXT", "Label", "Input_ID"]]

RANDOM_SEED = 42

original_df = data[1:100000]

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


print("chunk {} tokenize start!")
df_chunk = original_df.iloc[:100000].copy()
df_processed_chunk = preprocessing(df_chunk, tokenizer)
df_processed_chunk = df_processed_chunk.astype({'Adm_ID': 'int64', 'Note_ID': 'int64', 'Label': 'int64'})

df = pd.DataFrame({'Adm_ID': [], 'Note_ID': [], 'TEXT': [], 'Input_ID': [],
                       'Label': [], 'chartdate': [], 'charttime': []})
df = df.append(df_processed_chunk, ignore_index=True)

result = df.Label.value_counts()
    #write_log(
    #    "In the full dataset Positive Patients' Notes: {}, Negative Patients' Notes: {}".format(result[1],
    #                                                                                      result[0]),
    #    LOG_PATH)

dead_ID = pd.Series(df[df.Label == 1].Adm_ID.unique())
not_dead_ID = pd.Series(df[df.Label == 0].Adm_ID.unique())
    #write_log("Total Positive Patients' ids: {}, Total Negative Patients' ids: {}".format(len(dead_ID), len(not_dead_ID)), LOG_PATH)

#not_dead_ID_use = not_dead_ID.sample(n=args.id_num_neg, random_state=RANDOM_SEED)
#dead_ID_use = dead_ID.sample(n=args.id_num_pos, random_state=RANDOM_SEED)


id_val_test_t = dead_ID.sample(frac=0.2, random_state=RANDOM_SEED)
id_val_test_f = not_dead_ID.sample(frac=0.2, random_state=RANDOM_SEED)

id_train_t = dead_ID.drop(id_val_test_t.index)
id_train_f = not_dead_ID.drop(id_val_test_f.index)

id_val_t = id_val_test_t.sample(frac=0.5, random_state=RANDOM_SEED)
id_test_t = id_val_test_t.drop(id_val_t.index)
id_val_f = id_val_test_f.sample(frac=0.5, random_state=RANDOM_SEED)
id_test_f = id_val_test_f.drop(id_val_f.index)

id_test = pd.concat([id_test_t, id_test_f])
test_id_label = pd.DataFrame(data=list(zip(id_test, [1] * len(id_test_t) + [0] * len(id_test_f))),
                                     columns=['id', 'label'])

id_val = pd.concat([id_val_t, id_val_f])
val_id_label = pd.DataFrame(data=list(zip(id_val, [1] * len(id_val_t) + [0] * len(id_val_f))),
                                    columns=['id', 'label'])

id_train = pd.concat([id_train_t, id_train_f])
train_id_label = pd.DataFrame(data=list(zip(id_train, [1] * len(id_train_t) + [0] * len(id_train_f))),
                                      columns=['id', 'label'])

mortality_train = df[df.Adm_ID.isin(train_id_label.id)]
mortality_val = df[df.Adm_ID.isin(val_id_label.id)]
mortality_test = df[df.Adm_ID.isin(test_id_label.id)]
mortality_not_use = df[
            (~df.Adm_ID.isin(train_id_label.id)) & (~df.Adm_ID.isin(val_id_label.id) & (~df.Adm_ID.isin(test_id_label.id)))]

train_result = mortality_train.Label.value_counts()

val_result = mortality_val.Label.value_counts()

test_result = mortality_test.Label.value_counts()

no_result = mortality_not_use.Label.value_counts()


100%|██████████| 231508/231508 [00:00<00:00, 2315545.95B/s]


chunk {} tokenize start!


First sentence tokenized
['normal', 'sin', '##us', 'rhythm', '.', 'non', '-', 'diagnostic', 'rep', '##olar', '##ization', 'abnormalities', '.', 'compared', 'to', 'the', '##pre', '##vious', 'tracing', 'of', 'no', 'major', 'change', ',', 'although', 'non', '-', 'diagnostic', 'rep', '##olar', '##ization', 'abnormalities', 'are', 'now', 'evident', '.', 'tracing', '#', '2']


Streaming output truncated to the last 5000 lines.
<ipython-input-6-80e095cfcb3e>:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_processed_chunk, ignore_index=True)


In [ ]:
mortality_train.to_csv('/content/drive/My Drive/train.csv', index=False)
mortality_val.to_csv('/content/drive/My Drive/val.csv', index=False)
mortality_test.to_csv('/content/drive/My Drive/test.csv', index=False)
mortality_not_use.to_csv('/content/drive/My Drive/not_use.csv', index=False)
df.to_csv('/content/drive/My Drive/full.csv', index=False)

In [ ]:
df['Input_ID'] = mortality_train.Input_ID.apply(lambda x: ', '.join(map(str, x)))
df
#df['String_Column'] = mortality_val.Input_ID.apply(lambda x: ', '.join(map(str, x)))
#type(df['String_Column''])
#input_ids = df['String_Column'].apply(lambda x: x[1:-1].replace(' ', '').split(','))

0       [3671, 8254, 2271, 6348, 1012, 2512, 1011, 164...
1       [3671, 8254, 2271, 6348, 2302, 16474, 19470, 3...
2       [2340, 1024, 4720, 7610, 3108, 1006, 12109, 97...
3       [2184, 1024, 2539, 2572, 14931, 2132, 1059, 10...
4       [2184, 1024, 2871, 2572, 3108, 1006, 12109, 97...
                              ...                        
9994    [5776, 1013, 3231, 2592, 1024, 12407, 1024, 93...
9995    [8254, 2271, 11937, 11714, 11522, 2401, 3532, ...
9996    [8254, 2271, 6348, 1012, 28105, 2659, 10004, 1...
9997    [8254, 2271, 6348, 1012, 28105, 2659, 10004, 1...
9998    [1015, 1024, 4724, 7610, 20228, 11236, 2389, 2...
Name: Input_ID, Length: 7618, dtype: object